In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib2

ModuleNotFoundError: No module named 'urllib2'

In [ ]:
url = 'https://portal.amfiindia.com/DownloadNAVHistoryReport_Po.aspx?frmdt=23-Aug-2022'